In [49]:
import pandas as pd

df = pd.read_csv('lmgearnet_pipeline.csv')
df


,lm_layer,lm_train_layer,patience,sensitivity,specificity,accuracy,precision,mcc,micro_auroc,train_bce,valid_bce,valid_mcc,epoch_count
0,33,0,5,0.4785,0.9915,0.9650,0.7557,0.5849,0.9191,0.0008,0.3036,0.6254,25
1,33,1,5,0.5199,0.9931,0.9686,0.8049,0.6323,0.9229,0.0006,0.2254,0.6480,18
2,33,2,5,0.5981,0.9877,0.9675,0.7267,0.6426,0.9066,0.0018,0.1580,0.6687,13
3,33,0,5,0.5391,0.9875,0.9642,0.7012,0.5968,0.9308,0.0318,0.1283,0.6101,12
4,33,1,5,0.5183,0.9934,0.9688,0.8105,0.6337,0.9139,0.0007,0.2191,0.6401,16
5,33,2,5,0.5853,0.9901,0.9691,0.7630,0.6528,0.9218,0.0005,0.1824,0.6657,17
6,33,0,5,0.4561,0.9926,0.9648,0.7709,0.5770,0.9221,0.0024,0.2783,0.6202,24
7,33,1,5,0.5710,0.9919,0.9701,0.7938,0.6586,0.9245,0.0006,0.1990,0.6485,17
8,33,2,5,0.5598,0.9933,0.9708,0.8201,0.6637,0.9260,0.0060,0.1552,0.6643,11
9,33,3,5,0.6013,0.9911,0.9709,0.7871,0.6734,0.9077,0.0021,0.1864,0.6885,20


In [50]:
df_stat = df.groupby(['lm_layer', 'lm_train_layer', 'patience']).agg({
    'mcc': ['size', 'mean', 'std'],
       'sensitivity': ['mean', 'std'],
    #    'specificity': ['mean', 'std'],
    #    'accuracy': ['mean', 'std'],
       'precision': ['mean', 'std'],
       'valid_mcc': ['mean', 'std'],
       'epoch_count': ['mean', 'std']
    }
    ).reset_index()
df_stat

lm_layer lm_train_layer patience  mcc                     sensitivity  \
                                   size      mean       std        mean   
0       33              0        5    3  0.586233  0.009967    0.491233   
1       33              1        5    5  0.635340  0.015788    0.535220   
2       33              2        5    5  0.655520  0.008205    0.578620   
3       33              3        5    5  0.663920  0.013701    0.622980   
4       33              4        5    5  0.643180  0.024140    0.609880   
5       33              5        5    4  0.652200  0.008885    0.582950   
6       36              1        5    1  0.634100       NaN    0.527900   

            precision           valid_mcc           epoch_count            
        std      mean       std      mean       std        mean       std  
0  0.042940  0.742600  0.036650  0.618567  0.007780   20.333333  7.234178  
1  0.024334  0.791260  0.038546  0.645400  0.003917   14.800000  3.962323  
2  0.015062  0.777940  0.035766  0.669180  0.005595   14.200000  2.387467  
3  0.016189  0.742240  0.031492  0.670620  0.011420   15.400000  3.361547  
4  0.033913  0.717860  0.071781  0.680640  0.005979   14.000000  2.828427  
5  0.019517  0.765375  0.030139  0.692200  0.007357   17.000000  8.524475  
6       NaN  0.797600       NaN  0.654100       NaN   11.000000       NaN